In [ ]:
!pip install streamlit

In [5]:
from google.cloud import bigquery
from google.cloud.aiplatform_v1.types import NearestNeighborQuery
from vertexai.resources.preview import (FeatureOnlineStore, FeatureView,
                                        FeatureViewBigQuerySource)
from vertexai.resources.preview.feature_store import utils

#set project info
PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-central1" 

#Verify that the FeatureView instance is created by getting the feature view.
FEATURE_ONLINE_STORE_ID = "nine_quality_test_multimodal_featurestore"  # @param {type: "string"}
FEATURE_VIEW_ID = "feature_view_nine_quality_test"  # @param {type: "string"}
nine_fs=FeatureOnlineStore(FEATURE_ONLINE_STORE_ID)
nine_fv=FeatureView(
    FEATURE_VIEW_ID, feature_online_store_id=FEATURE_ONLINE_STORE_ID
) 
    

2024-08-27 12:27:15.220625: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-27 12:27:21.234298: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-08-27 12:27:21.236730: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [4]:
import streamlit as st
#from predict_request_gapic import *
import json
#import prediction client
from Embeddings.EmbeddingPredictionCLS import EmbeddingPredictionClient

st.set_page_config(
     layout="wide",
     page_title="JS Lab",
     page_icon="https://api.dicebear.com/5.x/bottts-neutral/svg?seed=gptLAb"
)

# Sidebar
st.sidebar.header("About")
st.sidebar.markdown(
    "9News Search Engine POC"
)

st.title("9News Search Engine: Semantic Searches for Images and Videos")
st.subheader("Implemented by Intelia")


#client to access multimodal-embeddings model to convert text to embeddings
client = EmbeddingPredictionClient(project=PROJECT_ID)


scopes = ["https://www.googleapis.com/auth/cloud-platform"]
sa_file_path = "serious-hall-371508-b584a49b4817.json"
##credentials = service_account.Credentials.from_service_account_file(sa_file_path, scopes=scopes)
##client_options = { "api_endpoint": "1256643097.asia-southeast1-712487506603.vdb.vertexai.goog"}

#client to access GCS bucket
##storage_client = storage.Client(credentials=credentials)
##bucket = storage_client.bucket("js-multimodal-embeddings")

 

allResults=[]

search_term = 'I am searching for ' + st.text_input('Search: ')
if search_term !='I am searching for ':
    converted_query_to_mm_embedding = client.get_multimodal_embedding(text=search_term).text_embedding
    converted_query_to_text_embedding = client.get_text_embedding(text=search_term).text_embedding
    nearest_neighbours=[]
    
    result=nine_fv.search(
        embedding_value=converted_query_to_mm_embedding,
        neighbor_count=10,
        #string_filters=[country_filter],#for multimodal embedding this can be set to None, unless having a description column
        return_full_entity=True,  # returning entities with metadata
    )
    result=result.to_dict()
    
    for neighbour in result['neighbors']:
        nearest_neighbour={}
        nearest_neighbour['entity_id']=neighbour['entity_id']
        nearest_neighbour['distance']=neighbour['distance']

        for feature in neighbour['entity_key_values']['key_values']['features']:
            if 'value' in feature:
                if type(list(feature['value'].values())[0]) is dict:
                    nearest_neighbour[feature['name']]=[]#list(list(feature['value'].values())[0].values())[0]             
                else:
                    nearest_neighbour[feature['name']]=list(feature['value'].values())[0]             
            else :
                nearest_neighbour[feature['name']]=None
            nearest_neighbour['embedding_type']='multimodal'

        nearest_neighbours.append(nearest_neighbour)


if len(nearest_neighbours)>=1:
 st.write("")
 st.write("These are the most relevant results matching your search query:")
 st.image(allResults, width=200)
elif search_term =='I am searching for ':
 st.write("Please type in a search query above")
else:
 st.write("Sorry! There are no images matching your query. Please try again.")

NameError: name 'service_account' is not defined

In [4]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 21.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 658.1/658.1 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 58.5 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 11.1 MB/s eta 0:00:00
